## Data Cleansing in Python with Pandas and Pyjanitor
Here's the code example for the blog post about [data cleaning using pandas](https://www.marsja.se/easiest-data-cleaning-method-using-python-pandas-pyjanitor/)

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from random import shuffle

import janitor

subject = ['n0' + str(i) for i in range(1, 201)]

# Generate response times  using a normal distribution
a = 457
rt = norm.rvs(a, size=200)

# Shuffle the response times 
shuffle(rt)

# Adding some missing data
rt[4], rt[9], rt[100] = np.nan, np.nan, np.nan


# Creating a dictionary
data = {
    'Subject': subject,
    'RT': rt,
}

# Pandas Dataframe from dictionary
df = pd.DataFrame(data)

df.head()

,Subject,RT
0,n01,455.476461
1,n02,456.811823
2,n03,456.155956
3,n04,457.285037
4,n05,NaN


### Append a Column, or Columns, to Pandas Dataframe:

#### Pandas Method

In [2]:
df['NewColumnName'] = np.nan
df.head()

,Subject,RT,NewColumnName
0,n01,455.476461,NaN
1,n02,456.811823,NaN
2,n03,456.155956,NaN
3,n04,457.285037,NaN
4,n05,NaN,NaN


#### Pyjanitor method:

In [3]:
newcolvals = [np.nan]*len(df['Subject'])
df = df.add_column('NewColumnName2', newcolvals)
df.head()

,Subject,RT,NewColumnName,NewColumnName2
0,n01,455.476461,NaN,NaN
1,n02,456.811823,NaN,NaN
2,n03,456.155956,NaN,NaN
3,n04,457.285037,NaN,NaN
4,n05,NaN,NaN,NaN


### Remove Missing Data

#### Pandas Dropna

In [4]:
df.dropna(subset=['RT']).head() # Set inplace=True to modify the dataframe

,Subject,RT,NewColumnName,NewColumnName2
0,n01,455.476461,NaN,NaN
1,n02,456.811823,NaN,NaN
2,n03,456.155956,NaN,NaN
3,n04,457.285037,NaN,NaN
5,n06,457.421874,NaN,NaN


In [5]:
### Pyjanitor
df.dropna(subset=['RT']).head()

,Subject,RT,NewColumnName,NewColumnName2
0,n01,455.476461,NaN,NaN
1,n02,456.811823,NaN,NaN
2,n03,456.155956,NaN,NaN
3,n04,457.285037,NaN,NaN
5,n06,457.421874,NaN,NaN


### Removing Empty Columns

#### Pandas Method

In [6]:
df.dropna(axis=1, how='all').head()

,Subject,RT
0,n01,455.476461
1,n02,456.811823
2,n03,456.155956
3,n04,457.285037
4,n05,NaN


#### Pyjanitor Method:

In [7]:
df.remove_empty().head()

,Subject,RT
0,n01,455.476461
1,n02,456.811823
2,n03,456.155956
3,n04,457.285037
4,n05,NaN


### Cleaning the Column Names

In [8]:
import requests
from pandas.io.json import json_normalize

url = "https://datahub.io/core/s-and-p-500-companies-financials/r/constituents-financials.json"
resp = requests.get(url=url)

df = json_normalize(resp.json())
df.iloc[:,0:6].head()

,52 Week High,52 Week Low,Dividend Yield,EBITDA,Earnings/Share,Market Cap
0,175.490,259.77,2.332862,9.048000e+09,7.92,1.387211e+11
1,48.925,68.39,1.147959,6.010000e+08,1.70,1.078342e+10
2,42.280,64.60,1.908982,5.744000e+09,0.26,1.021210e+11
3,60.050,125.86,2.499560,1.031000e+10,3.29,1.813863e+11
4,114.820,162.60,1.714470,5.643228e+09,5.44,9.876586e+10


#### Using Pandas Rename

In [9]:
import re

df.rename(columns=lambda x: re.sub('(\s|/)','_',x), 
          inplace=True)
df.keys()

Index(['52_Week_High', '52_Week_Low', 'Dividend_Yield', 'EBITDA',
       'Earnings_Share', 'Market_Cap', 'Name', 'Price', 'Price_Book',
       'Price_Earnings', 'Price_Sales', 'SEC_Filings', 'Sector', 'Symbol'],
      dtype='object')

#### Using clean_names from Pyjanitor

In [10]:
df = df.clean_names().head()
df.keys()

Index(['52_week_high', '52_week_low', 'dividend_yield', 'ebitda',
       'earnings_share', 'market_cap', 'name', 'price', 'price_book',
       'price_earnings', 'price_sales', 'sec_filings', 'sector', 'symbol'],
      dtype='object')

### Grouping and Aggregating:

In [11]:
df.groupby('sector').agg(['mean',
                          'std']).collapse_levels().reset_index()


,sector,52_week_high_mean,52_week_high_std,52_week_low_mean,52_week_low_std,dividend_yield_mean,dividend_yield_std,ebitda_mean,ebitda_std,earnings_share_mean,...,market_cap_mean,market_cap_std,price_mean,price_std,price_book_mean,price_book_std,price_earnings_mean,price_earnings_std,price_sales_mean,price_sales_std
0,Health Care,51.1650,12.565288,95.23,43.317361,2.204271,0.417601,8.027000e+09,3.228650e+09,1.775,...,1.417537e+11,5.604903e+10,82.375,36.918045,14.665,16.228101,20.960,2.192031,5.016026,1.803893
1,Industrials,112.2075,89.494970,164.08,135.326096,1.740410,0.837853,4.824500e+09,5.972931e+09,4.810,...,7.475224e+10,9.046557e+10,141.565,115.010918,8.845,3.528463,26.035,2.439518,3.982877,0.576142
2,Information Technology,114.8200,NaN,162.60,NaN,1.714470,NaN,5.643228e+09,NaN,5.440,...,9.876586e+10,NaN,150.510,NaN,10.620,NaN,25.470,NaN,2.604117,NaN


### All together now!
Here we use Pyjanitor for all the above tasks (except for summary stats, of course).

In [12]:
data_id = [1]*200
df = (
    pd.read_csv('./SimData/DF_NA_Janitor.csv', 
                index_col=0)
    .add_column('data_id', data_id)
    .remove_empty()
    .dropna()
    .clean_names()
)

df.head()

,subject_id,first_name,day,age,response_time,gender,data_id
0,1,John,Sixth,23,0.562719,0,1
3,4,Don,Sixth,27,0.522961,0,1
5,6,James,Sixth,25,0.457103,0,1
11,12,Eric,Sixth,23,0.512688,0,1
13,14,James,Sixth,21,0.451782,0,1
